In [ ]:
import numpy as np
 
IMG_SIZE_W = 224
IMG_SIZE_H = 224
LR = 1e-3
 
training_data = np.load('data_with_label.npy',allow_pickle=True)
x_train=np.array([i[0] for i in training_data]).reshape(-1,IMG_SIZE_W,IMG_SIZE_H ,1)
y_train=np.array([i[1] for i in training_data]).reshape(-1,1)
x_train=x_train/255
 
testing_data = np.load('data_with_label.npy',allow_pickle=True)
x_test=np.array([i[0] for i in testing_data]).reshape(-1,IMG_SIZE_W,IMG_SIZE_H,1)
y_test=np.array([i[1] for i in testing_data]).reshape(-1,1)
x_test=x_test/255
 
MODEL_NAME = 'anyname-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match
 
 
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
tf.reset_default_graph()
 
 
convnet = input_data(shape=[None, IMG_SIZE_W, IMG_SIZE_H, 1], name='input')
 
convnet = conv_2d(convnet, 256, 4, activation='relu')
convnet = max_pool_2d(convnet, 4)
 
convnet = conv_2d(convnet, 128, 4, activation='relu')
convnet = max_pool_2d(convnet, 4)
 
convnet = conv_2d(convnet, 64, 4, activation='relu')
convnet = max_pool_2d(convnet, 4)
 
convnet = conv_2d(convnet, 32, 4, activation='relu')
convnet = max_pool_2d(convnet, 4)
 
convnet = conv_2d(convnet, 16, 4, activation='relu')
convnet = max_pool_2d(convnet, 4)
 
 
convnet = fully_connected(convnet, 128, activation='relu')
convnet = dropout(convnet, 0.5)
 
convnet = fully_connected(convnet, 1, activation='sigmoid')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
 
model = tflearn.DNN(convnet, tensorboard_dir='log')
 
 
 
model.fit({'input': x_train}, {'targets': y_train}, n_epoch=30, validation_set=({'input': x_test}, {'targets': y_test}),
          snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
 
model.save(MODEL_NAME)